# Setup Comprehend Through AWS Console 

<img src="img/comprehend.png" width="80%" align="left">

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(
   retries = {
      'max_attempts': 10,
      'mode': 'adaptive'
   }
)

iam = boto3.client('iam', config=config)
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
comprehend = boto3.client('comprehend')

In [3]:
%store -r noheader_train_s3_uri

print(noheader_train_s3_uri)

s3://sagemaker-us-west-2-393371431575/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [4]:
!aws s3 ls $noheader_train_s3_uri

2020-07-25 15:59:25   13643009 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


# Create Data Access Role for Comprehend

## Create Policy

In [5]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [6]:
iam_comprehend_role_name = 'DSOAWS_Comprehend'

In [7]:
import json
import time

from botocore.exceptions import ClientError

try:
    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(30)

In [8]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)

{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-393371431575/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-393371431575'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-393371431575/*'], 'Effect': 'Allow'}]}


# Attach Policy to Role

In [9]:
import time

response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='DSOAWS_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '77d3fb10-7fca-4a2d-b95b-cfb78f76523e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '77d3fb10-7fca-4a2d-b95b-cfb78f76523e', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sat, 25 Jul 2020 16:30:25 GMT'}, 'RetryAttempts': 0}}


# Train the Model

In [10]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-us-west-2-393371431575/models/comprehend/output


In [11]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [12]:
import datetime
import time

id = str(datetime.datetime.now().strftime("%s"))

training_job = comprehend.create_document_classifier(
    DocumentClassifierName='Amazon-Customer-Reviews-Classifier-'+ id,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

time.sleep(30)

In [13]:
import time

jobArn = training_job['DocumentClassifierArn']

max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = jobArn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(10)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom cla

# _Please Wait Until the ^^ Classifier ^^ is Trained Above._

# Show Results of the Classifier

In [14]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-west-2:393371431575:document-classifier/Amazon-Customer-Reviews-Classifier-1595694655', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 7, 25, 16, 30, 55, 845000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 7, 25, 17, 18, 15, 416000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 7, 25, 16, 34, 53, 912000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 7, 25, 17, 16, 20, 326000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-393371431575/models/comprehend/output/393371431575-CLR-55f759fa55e85f3317fced5dccda71ec/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 27905, 'NumberOfTestDocuments': 3100, 'EvaluationMetrics': {'Accuracy': 0.5477, 'Precision': 0.5457, 'Rec

In [15]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-west-2:393371431575:document-classifier/Amazon-Customer-Reviews-Classifier-1595694655


In [16]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-us-west-2-393371431575/models/comprehend/output/393371431575-CLR-55f759fa55e85f3317fced5dccda71ec/output/output.tar.gz
models/comprehend/output/393371431575-CLR-55f759fa55e85f3317fced5dccda71ec/output/output.tar.gz


In [17]:
#Download the model metrics
s3 = boto3.resource('s3')

s3.Bucket(bucket).download_file(job_key, './output.tar.gz')

In [18]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [19]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      398,
      142,
      47,
      15,
      18
    ],
    [
      144,
      272,
      171,
      23,
      10
    ],
    [
      59,
      128,
      295,
      109,
      29
    ],
    [
      18,
      43,
      103,
      298,
      158
    ],
    [
      13,
      10,
      25,
      137,
      435
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [20]:
!pip install tabulate

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [21]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,398,142,47,15,18,
2,144,272,171,23,10,
3,59,128,295,109,29,
4,18,43,103,298,158,
5,13,10,25,137,435,
(Actual),,,,,,


# Deploy Endpoint

In [22]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

comprehend_endpoint_name = 'comprehend-inference-ep-' + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name,
    ModelArn = model_arn,
    DesiredInferenceUnits = 1
)

In [23]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-west-2:393371431575:document-classifier-endpoint/comprehend-inference-ep-25-17-18-27


# Pass Variables to the Next Notebook(s)

In [24]:
%store comprehend_endpoint_arn

Stored 'comprehend_endpoint_arn' (str)


# Predict with Endpoint

In [25]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = comprehend_endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-west-2:393371431575:document-classifier-endpoint/comprehend-inference-ep-25-17-18-27', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-west-2:393371431575:document-classifier/Amazon-Customer-Reviews-Classifier-1595694655', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 7, 25, 17, 18, 27, 369000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 7, 25, 17, 18, 27, 369000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '8f340456-45b6-4574-a648-211df4693a19', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8f340456-45b6-4574-a648-211df4693a19', 'content-type': 'application/x-amz-json-1.1', 'content-length': '408', 'date': 'Sat, 25 Jul 2020 17:18:26 GMT'}, 'RetryAttempts': 0}}


In [26]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = comprehend_endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

In [27]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.891700029373169
    },
    {
      "Name": "4",
      "Score": 0.07680000364780426
    },
    {
      "Name": "1",
      "Score": 0.01209999993443489
    }
  ],
  "ResponseMetadata": {
    "RequestId": "8b4985f4-01b8-4992-b723-7eea3f56cc96",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "8b4985f4-01b8-4992-b723-7eea3f56cc96",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "134",
      "date": "Sat, 25 Jul 2020 17:27:19 GMT"
    },
    "RetryAttempts": 0
  }
}


In [28]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.6717000007629395
    },
    {
      "Name": "4",
      "Score": 0.13670000433921814
    },
    {
      "Name": "2",
      "Score": 0.12330000102519989
    }
  ],
  "ResponseMetadata": {
    "RequestId": "73d85ce9-e7b0-47e7-9e2e-1978458f0ca1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "73d85ce9-e7b0-47e7-9e2e-1978458f0ca1",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 25 Jul 2020 17:27:19 GMT"
    },
    "RetryAttempts": 0
  }
}


In [29]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.45399999618530273
    },
    {
      "Name": "2",
      "Score": 0.3050000071525574
    },
    {
      "Name": "3",
      "Score": 0.18569999933242798
    }
  ],
  "ResponseMetadata": {
    "RequestId": "727658a2-d3f0-4b98-bd14-77036a43fdca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "727658a2-d3f0-4b98-bd14-77036a43fdca",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 25 Jul 2020 17:27:19 GMT"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

# Navigate to Comprehend in the AWS Console
![Comprehend Console](img/comprehend-console.png)